## Import Dependencies


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import pandas as pd
import os
from PIL import Image

## Create Dataset for local Directories

In [29]:
class CIFAKE_Dataset(Dataset):
    def __init__(self, root_dir, transform=torchvision.transforms.ToTensor()):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['REAL', 'FAKE']
        
    def __len__(self):
        total_len = 0
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            total_len += len(os.listdir(class_dir))
        return total_len
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            class_files = os.listdir(class_dir)
            if idx < len(class_files):
                img_name = os.path.join(class_dir, class_files[idx])
                image = Image.open(img_name)
                label = self.classes.index(cls)
                if self.transform:
                    image = self.transform(image)
                return image, label
            else:
                idx -= len(class_files)

### Load the dataset

In [30]:
# Load the dataset from local directory without preprocessing
training_data = CIFAKE_Dataset(root_dir='data/train')
testing_data = CIFAKE_Dataset(root_dir='data/test')
image_datasets = {'train': training_data, 'val': testing_data}

In [31]:
# Defind the data loader
train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=64, shuffle=True)
dataloaders = {'train': train_loader, 'test': test_loader}

In [32]:
num_classes = 2
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
    nn.Softmax(dim=1)
)

/home/beik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/beik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/beik/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 29.5MB/s]


In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [34]:
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [35]:
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        running_loss = 0.0
        corrects = 0
        
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
            running_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = corrects.double() / len(image_datasets[phase])
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

# Save the trained model
torch.save(model.state_dict(), 'resnet50_finetuned.pth')

KeyboardInterrupt: 

##别动电脑，在训练模型